# Уравнивание теодолитного хода  

In [4]:
from sympy import*
from sympy.abc import*
import numpy as np
from math import*
# Исходные данные
nish=4
nnapr=8
nst=7
ish=[[10026.767,10162.985],[10794.135,9484.527],[13024.727,7935.593],[14282.006,7745.765]]
napr=[[159.10252],[189.44214],[206.02405],[154.01152],[209.38185],[190.09070],[195.51544],[168.15416]]
st=[[413.311,1],[528.497,1],[395.857,1],[434.657,1],[406.105,1],[452.050,1],[283.786,1]]
mu=5
mb=2
# Преобразрвание угда в градусы
def grad(napr):
    seconds=(napr*100-int(napr*100))*100/3600
    minutes=int((napr-int(napr))*100)/60
    nap1=int(napr)+minutes+seconds
    return(nap1)
# Огз
def ogz(x1,x2,y1,y2):
    a=0
    pr1=(x1-x2)
    pr2=(y1-y2)
    x=abs((atan(pr2/pr1)))
    r=x*180/pi #перевод в градусы
    if pr1>0 and pr2>0:
        a+=r
        a=a*pi/180
        f=1
    elif pr1<0 and pr2>0:
        a+=180-r
        f=2
        a=a*pi/180
    elif pr1 <0 and pr2<0:
        a+=180+r
        f=3
        a=a*pi/180
    elif pr1>0 and pr2<0:
        a+=360-r
        f=4
        a=a*pi/180
    else:
        print("выбраны неверные данные")
    S=sqrt(pr1**2+pr2**2)
    return(a,f,S)
# Пгз
def pgz(x2,y2,an,d):
    dx=d*cos(an)
    dy=d*sin(an)
    X=(x2+dx)
    Y= (y2+dy)
    return(X,Y)
# Определение предварительных координат, определяемых пунктов
i=0
j=0
kop=[(ish[i+1][j],ish[i+1][j+1])]
x1=ish[i][j]
y1=ish[i][j+1]
x2=ish[i+1][j]
y2=ish[i+1][j+1]
for i in range(nnapr-1):
    d=st[i][j]
    av= grad(napr[i][j])
    og=ogz(x1,x2,y1,y2)
    a=og[0]
    f=og[1]
    if f==1:
        an=av+a
    else:
        an=av*pi/180-(360*pi/180-a)
    pg= pgz(x2,y2,an,d)
    kop+=[pg]
    if i==0:
        x1=ish[i+1][j]
        y1=ish[i+1][j+1]
        x2=kop[i+1][j]
        y2=kop[i+1][j+1]
    elif i>0:
        x1=kop[i][j]
        y1=kop[i][j+1]
        x2=kop[i+1][j]
        y2=kop[i+1][j+1]  
i=0
kop+=[(ish[-i-1][j],ish[-i-1][j+1])]
# 1
OG=[]
i=0
j=0
for i in range(nnapr):
    x1=kop[i][j]
    y1=kop[i][j+1]
    x2=kop[i+1][j]
    y2=kop[i+1][j+1]
    og=ogz(x1,y1,x2,y2)
    OG+=[og]
m=len(OG)-2
# 2
# Матрица дирекционных углов
alf=[]
mst=[]
i=0
j=0
for i in range(nnapr):
    alf+=[[OG[i][j]]]
# Матрица сторон
i=0
for i in range(nnapr-1):
    mst+=[[OG[i][j+2]*100]]
# Матрица коэффициентов параметрических уравнений поправок (A):
A=[]
i=0
j=0
k=0
for i in range(nst):
    yrst=[]
    for k in range(m):
        if k==i and k!=0 and i!=0:
            yrs=[-cos(alf[i][j]),-sin(alf[i][j])]
        elif k==i+1:
            yrs=[cos(alf[i][j]),sin(alf[i][j])]
        elif i==0 and k==0:
            yrs=[cos(alf[i][j]),sin(alf[i][j])]
        else:
            yrs=[0,0]
        yrst+=yrs
    A+=[yrst]
i=0
j=0
k=0
for i in range(nnapr):
    yrnapr=[]
    for k in range(m):
        if k==i:
            if i==0 and k==0:
                 yrn=[-sin(alf[i][j])/mst[i][j],cos(alf[i][j])/mst[i][j]]
            else:
                yrn=[sin(alf[i][j])/mst[i][j],-cos(alf[i][j])/mst[i][j]]
        elif k==i+1:
            yrn=[-sin(alf[i][j])/mst[i][j],cos(alf[i][j])/mst[i][j]]
        else:
            yrn=[0,0]
        yrnapr+=yrn
    A+=[yrnapr]
A=np.array(A)
At=A.transpose()
# Матрица весов (P):
P=[]
i=0

for i in range(nst):
    ps=[(mu**2)/(st[i][j+1])**2]
    P+=ps
for i in range(nnapr):
    pb=[(mu**2)/mb**2]
    P+=pb
pd=np.diag(P)
# Матрица свободных членов (L):
L=[]
i=0
for i in range(nst):
    ls=[mst[i][j]-st[i][j]]
    L+=ls
for i in range(nnapr):
    lm=[alf[i][j]-napr[i][j]]
    L+=lm
# Транспонирование
Lt=[]
i=0
for i in range(nst):
    ls=[mst[i][j]-st[i][j]]
    Lt+=[ls]
for i in range(nnapr):
    lm=[alf[i][j]-napr[i][j]]
    Lt+=[lm]
# Матрица коэффициентов нормальных уравнений (N):
at=np.matrix(At)
B = np.matrix(pd)
C = at.dot(B)
N=C.dot(A)
# Матрица свободных членов нормальных уравнений (W):
at = np.matrix(At)
l=np.matrix(Lt)
B = np.matrix(pd)
C = at.dot(B)
W=C.dot(l)
Wt=W.transpose()
# Матрица весовых коэффицикнтов (Q):
n=np.array(N)
G=np.linalg.inv(n)
# Вычисление поправок в предварительные значения координат:
g = np.matrix(G)
w=np.matrix(Wt)
T=-w.dot(g)
Tt=T.transpose()
# Вычисление поправок в измерения:
t = np.matrix(Tt)
a=np.matrix(A)
V=a.dot(t)+l
Vt=V.transpose()
# Вычисление апостериорной погрешности:
vt=np.matrix(Vt)
v1=np.matrix(V)
v=v1/3
B1 = np.matrix(pd)
VT=vt.dot(B1)
VTV=VT.dot(v)
mu1=VTV
mu=sqrt(mu1)
print(mu)

19421.465499147496
